In [1]:
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt

# register at https://scihub.copernicus.eu/dhus/#/home
user = ''
password = ''
api = SentinelAPI(user, password, 'https://scihub.copernicus.eu/dhus')

In [2]:
footprint = geojson_to_wkt(read_geojson('../glass-fire/map.geojson'))


products = api.query(
    footprint, date=('20200920', "20201001"), platformname='Sentinel-2', producttype="S2MSI1C"
)

product_df = api.to_dataframe(products)
product_df.head()

,title,link,link_alternative,link_icon,summary,ondemand,datatakesensingstart,beginposition,endposition,ingestiondate,...,s2datatakeid,producttype,platformidentifier,orbitdirection,platformserialidentifier,processinglevel,identifier,uuid,granuleidentifier,datastripidentifier
f48fb0ee-a209-4b66-b5ec-5c3a66190412,S2B_MSIL1C_20200929T190109_N0209_R013_T10SDJ_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2020-09-29T19:01:09.024Z, Instrument: MS...",false,2020-09-29 19:01:09.024,2020-09-29 19:01:09.024,2020-09-29 19:01:09.024,2020-09-29 22:30:30.319,...,GS2B_20200929T190109_018629_N02.09,S2MSI1C,2017-013A,DESCENDING,Sentinel-2B,Level-1C,S2B_MSIL1C_20200929T190109_N0209_R013_T10SDJ_2...,f48fb0ee-a209-4b66-b5ec-5c3a66190412,S2B_OPER_MSI_L1C_TL_VGS1_20200929T213343_A0186...,S2B_OPER_MSI_L1C_DS_VGS1_20200929T213343_S2020...
71d5a022-f6ad-4cbc-ab41-ba782a8481ee,S2B_MSIL1C_20200929T190109_N0209_R013_T10SEJ_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2020-09-29T19:01:09.024Z, Instrument: MS...",false,2020-09-29 19:01:09.024,2020-09-29 19:01:09.024,2020-09-29 19:01:09.024,2020-09-29 22:30:17.643,...,GS2B_20200929T190109_018629_N02.09,S2MSI1C,2017-013A,DESCENDING,Sentinel-2B,Level-1C,S2B_MSIL1C_20200929T190109_N0209_R013_T10SEJ_2...,71d5a022-f6ad-4cbc-ab41-ba782a8481ee,S2B_OPER_MSI_L1C_TL_VGS1_20200929T213343_A0186...,S2B_OPER_MSI_L1C_DS_VGS1_20200929T213343_S2020...
bd491342-5a1b-4e93-846b-74d4052bdfbe,S2B_MSIL1C_20200929T190109_N0209_R013_T10SDH_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2020-09-29T19:01:09.024Z, Instrument: MS...",false,2020-09-29 19:01:09.024,2020-09-29 19:01:09.024,2020-09-29 19:01:09.024,2020-09-29 22:28:26.919,...,GS2B_20200929T190109_018629_N02.09,S2MSI1C,2017-013A,DESCENDING,Sentinel-2B,Level-1C,S2B_MSIL1C_20200929T190109_N0209_R013_T10SDH_2...,bd491342-5a1b-4e93-846b-74d4052bdfbe,S2B_OPER_MSI_L1C_TL_VGS1_20200929T213343_A0186...,S2B_OPER_MSI_L1C_DS_VGS1_20200929T213343_S2020...
f060901c-8bf6-47c4-be2f-fd7a253e7a91,S2B_MSIL1C_20200929T190109_N0209_R013_T10SEH_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2020-09-29T19:01:09.024Z, Instrument: MS...",false,2020-09-29 19:01:09.024,2020-09-29 19:01:09.024,2020-09-29 19:01:09.024,2020-09-29 22:28:17.463,...,GS2B_20200929T190109_018629_N02.09,S2MSI1C,2017-013A,DESCENDING,Sentinel-2B,Level-1C,S2B_MSIL1C_20200929T190109_N0209_R013_T10SEH_2...,f060901c-8bf6-47c4-be2f-fd7a253e7a91,S2B_OPER_MSI_L1C_TL_VGS1_20200929T213343_A0186...,S2B_OPER_MSI_L1C_DS_VGS1_20200929T213343_S2020...
33cc42a2-3f3b-4a1c-bcdd-b4ea19d753ee,S2B_MSIL1C_20200926T185049_N0209_R113_T10SEJ_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2020-09-26T18:50:49.024Z, Instrument: MS...",false,2020-09-26 18:50:49.024,2020-09-26 18:50:49.024,2020-09-26 18:50:49.024,2020-09-26 23:20:57.362,...,GS2B_20200926T185049_018586_N02.09,S2MSI1C,2017-013A,DESCENDING,Sentinel-2B,Level-1C,S2B_MSIL1C_20200926T185049_N0209_R113_T10SEJ_2...,33cc42a2-3f3b-4a1c-bcdd-b4ea19d753ee,S2B_OPER_MSI_L1C_TL_VGS2_20200926T210259_A0185...,S2B_OPER_MSI_L1C_DS_VGS2_20200926T210259_S2020...


In [3]:
## A product is only available within a year on the server. After that it needs to be requested. Only one request per day...
# for product_id in products:
#     product_info = api.get_product_odata(product_id)
#     print(product_id, product_info["Online"])

In [4]:
api.download_all(products, directory_path='glass-fire/data/')

Downloading:   0%|          | 0.00/725M [00:00<?, ?B/s]
Downloading:   0%|          | 1.05M/725M [00:00<03:03, 3.94MB/s]
Downloading:   0%|          | 3.15M/725M [00:00<03:29, 3.45MB/s]
Downloading:   1%|          | 5.24M/725M [00:01<03:20, 3.59MB/s]
Downloading:   1%|          | 6.29M/725M [00:01<03:19, 3.60MB/s]
Downloading:   1%|          | 8.39M/725M [00:02<03:09, 3.77MB/s]
Downloading:   1%|▏         | 9.44M/725M [00:02<03:04, 3.89MB/s]
Downloading:   2%|▏         | 11.5M/725M [00:03<02:58, 4.01MB/s]
Downloading:   2%|▏         | 12.6M/725M [00:03<03:03, 3.89MB/s]
Downloading:   2%|▏         | 13.6M/725M [00:03<03:00, 3.95MB/s]
Downloading:   2%|▏         | 15.7M/725M [00:04<02:55, 4.04MB/s]
Downloading:   2%|▏         | 16.8M/725M [00:04<03:03, 3.87MB/s]
Downloading:   2%|▏         | 17.8M/725M [00:04<03:07, 3.77MB/s]
Downloading:   2%|▏         | 12.6M/527M [00:04<02:35, 3.32MB/s]
Downloading:   3%|▎         | 18.9M/725M [00:05<03:19, 3.55MB/s]
Downloading:   3%|▎         | 19.9

({'71d5a022-f6ad-4cbc-ab41-ba782a8481ee': {'id': '71d5a022-f6ad-4cbc-ab41-ba782a8481ee',
   'title': 'S2B_MSIL1C_20200929T190109_N0209_R013_T10SEJ_20200929T213343',
   'size': 526964501,
   'md5': '453B500BEB905E48F00E4DCF3E988182',
   'date': datetime.datetime(2020, 9, 29, 19, 1, 9, 24000),
   'footprint': 'POLYGON((-122.45613 39.747265402797574,-122.26953 39.703905469909046,-122.26947 39.70411470119142,-122.26915 39.70404232432752,-122.26868 39.70543046890472,-122.257126 39.702725704572174,-122.25722 39.70244204261585,-122.05838 39.652641878803536,-121.97246 39.630817448902064,-122.02365 39.48548261346787,-122.07518 39.338995955878815,-122.12665 39.19251354944865,-122.178116 39.04606428652325,-122.229385 38.8996087508536,-122.27905 38.75696732563208,-123.00023 38.760864457162924,-123.00023 39.7502679270046,-122.45613 39.747265402797574))',
   'url': "https://scihub.copernicus.eu/dhus/odata/v1/Products('71d5a022-f6ad-4cbc-ab41-ba782a8481ee')/$value",
   'Online': True,
   'Creation Da